In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hp2chamberofsecrets/6TheHalfBloodPrince.txt
/kaggle/input/hp2chamberofsecrets/2ChamberofSecrets.txt
/kaggle/input/hp2chamberofsecrets/1SorcerersStone.txt
/kaggle/input/hp2chamberofsecrets/5OrderofthePhoenix.txt
/kaggle/input/hp2chamberofsecrets/3ThePrisonerOfAzkaban.txt
/kaggle/input/hp2chamberofsecrets/7DeathlyHollows.txt
/kaggle/input/hp2chamberofsecrets/4TheGobletOfFire.txt


In [2]:
import tensorflow as tf
import numpy as np
import os
import time

In [3]:
files= ['/kaggle/input/hp2chamberofsecrets/1SorcerersStone.txt', '/kaggle/input/hp2chamberofsecrets/2ChamberofSecrets.txt', '/kaggle/input/hp2chamberofsecrets/3ThePrisonerOfAzkaban.txt', '/kaggle/input/hp2chamberofsecrets/4TheGobletOfFire.txt', '/kaggle/input/hp2chamberofsecrets/5OrderofthePhoenix.txt', '/kaggle/input/hp2chamberofsecrets/6TheHalfBloodPrince.txt', '/kaggle/input/hp2chamberofsecrets/7DeathlyHollows.txt']
with open('harrypotter.txt', 'w') as outfile:
  for file in files:
    with open(file) as infile:
      outfile.write(infile.read())

text = open('harrypotter.txt').read()
print ('Length of text: {} characters'.format(len(text)))

Length of text: 6251651 characters


In [4]:
# Taking a look at the text
print(text[:300])

Harry Potter and the Sorcerer's Stone 

CHAPTER ONE 

THE BOY WHO LIVED 

Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much. They were the last people you'd expect to be involved in anything strange or mysterious, because they 


In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

106 unique characters


In [6]:
# Creating a mapping from unique characters to indices
char2index = {u:i for i, u in enumerate(vocab)}
index2char = np.array(vocab)

text_as_int = np.array([char2index[c] for c in text])

print(text_as_int)

[39 64 81 ... 75 75 15]


In [7]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} -- characters mapped to int -- > {}'.format(repr(text[:13]), text_as_int[:13]))

'Harry Potter ' -- characters mapped to int -- > [39 64 81 81 88  3 47 78 83 83 68 81  3]


In [8]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(index2char[i.numpy()])


H
a
r
r
y


In [9]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(index2char[item.numpy()])))



"Harry Potter and the Sorcerer's Stone \n\nCHAPTER ONE \n\nTHE BOY WHO LIVED \n\nMr. and Mrs. Dursley, of nu"
'mber four, Privet Drive, were proud to say that they were perfectly normal, thank you very much. They'
" were the last people you'd expect to be involved in anything strange or mysterious, because they jus"
"t didn't hold with such nonsense. \n\nMr. Dursley was the director of a firm called Grunnings, which ma"
'de drills. He was a big, beefy man with hardly any neck, although he did have a very large mustache. '


In [10]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [11]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [12]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 300 #256

# Number of RNN units
rnn_units1 = 1024
rnn_units2 = 1024
rnn_units=[rnn_units1, rnn_units2]
print(vocab_size)

106


In [13]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units1,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.GRU(rnn_units2,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [14]:
model = build_model(
  vocab_size = vocab_size,
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 300)           31800     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          4073472   
_________________________________________________________________
gru_1 (GRU)                  (64, None, 1024)          6297600   
_________________________________________________________________
gru_2 (GRU)                  (64, None, 512)           2362368   
_________________________________________________________________
dense (Dense)                (64, None, 106)           54378     
Total params: 12,819,618
Trainable params: 12,819,618
Non-trainable params: 0
_________________________________________________________________


In [16]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [17]:
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

In [18]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [19]:
EPOCHS=20

In [20]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
967/967 [==============================] - 118s 122ms/step - loss: 1.5973 - accuracy: 0.5394
Epoch 2/20
967/967 [==============================] - 118s 122ms/step - loss: 1.2154 - accuracy: 0.6327
Epoch 3/20
967/967 [==============================] - 118s 122ms/step - loss: 1.1615 - accuracy: 0.6469
Epoch 4/20
967/967 [==============================] - 118s 122ms/step - loss: 1.1328 - accuracy: 0.6549
Epoch 5/20
967/967 [==============================] - 117s 121ms/step - loss: 1.1135 - accuracy: 0.6601
Epoch 6/20
967/967 [==============================] - 117s 121ms/step - loss: 1.0994 - accuracy: 0.6640
Epoch 7/20
967/967 [==============================] - 117s 121ms/step - loss: 1.0895 - accuracy: 0.6668
Epoch 8/20
967/967 [==============================] - 117s 121ms/step - loss: 1.0822 - accuracy: 0.6688
Epoch 9/20
967/967 [==============================] - 117s 121ms/step - loss: 1.0764 - accuracy: 0.6706
Epoch 10/20
967/967 [==============================] - 117s 121m

In [21]:
latest_check= tf.train.latest_checkpoint(checkpoint_dir)

In [22]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(latest_check)

model.build(tf.TensorShape([1, None]))

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 300)            31800     
_________________________________________________________________
gru_3 (GRU)                  (1, None, 1024)           4073472   
_________________________________________________________________
gru_4 (GRU)                  (1, None, 1024)           6297600   
_________________________________________________________________
gru_5 (GRU)                  (1, None, 512)            2362368   
_________________________________________________________________
dense_1 (Dense)              (1, None, 106)            54378     
Total params: 12,819,618
Trainable params: 12,819,618
Non-trainable params: 0
_________________________________________________________________


In [24]:
def generate_text(model, start_string):

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2index[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low results in more predictable text.
  # Higher results in more surprising text.
  # Experiment to find the best setting.
  scaling = 0.5 #1

  # batch size == 1
  
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / scaling
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(index2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [25]:
print(generate_text(model, start_string=u"Dumbledore "))

Dumbledore had wanted to go back to her father and soul, and you know I haven't think we were so long as he says he's gone and he was not the only one.
"You listen to your broomstick, and the werewolf had been here at the last of the Death Eaters had been able to find a long time at all. He was the only one he was wrong with the contents of the class and a group of conduction on her hand the second floor with a friendly undertone good in a long dark card from the time that it was worthless and position, the floor had been the only ones who had been the one who had insed one another word. He looked around the marble, and the scream of flames the parchment and the crowd took out a book of flowered cloaks, and the others and examined the gargoyle to the grounds. "The Prophet has come to a spell and can do it, he thought you'd been a big wand at the tournament of the wizard."

"And what are you doing what?" said Harry, turning the fake wand. "I thought you got to his feet the first thing t

In [28]:
print(generate_text(model, start_string=u"Harry killed Ron  "))

Harry killed Ron  and he jumped as he had decided that the three body else had flitted and he was sure that the world was thinking that he was safe. The child was seeing the process of the search of the castle, and whether you can accept that the snake was brandishing his stomach and the only words he had never seen him and he had seen in the forest, and he could hear her alone, so that the starry sky let out a few moments of the banister and the scratching of red lay the other side of the room.
"He do," said Dumbledore. "I was a great serpents and the process that was the first time for the portrait of the ground as he could not believe that it was a pretty good look.
"The Death Eaters have to kill them when I must be a protection," said Ron, looking down at the door and the more to help him bending the moment they had been achieved in the darkness. Harry saw that he was really lived about the moment or two four feet away. He was standing at a small look of what looked like a dream, a

In [29]:
print(generate_text(model, start_string=u"Harry killed Ron  "))

Harry has elderwand and drew his voice from the window. He saw a few seconds before he could recognize the newspaper to Harry. The last time he had found the connection was on the branches of the edging statue that nobody else was closed behind him as the three of them had the last time he had not gone and say he can have to say that. You see, he's coming to make our prisoners that was concerned. He stood up, looking at him the chair in the corner that was the sign of a great surprise, and the other the second of the finer languages were rang out of the edge of the ground and he could not see anything to discover the tops of his mouth with a bang and he was glad to come to the corner, and as he looked at each other. The wizard could hardly scream and cast into the middle to the pont of cold and shabby ranks and shallow seemed to be the sight of the bank of a corner, and a shower of laughter and the Death Eaters were the onlooks.
Snape seemed to have found the others and the trees to hi

In [30]:
print(generate_text(model, start_string=u"Hermoine Killed Snape  "))

Hermoine Killed Snape  of the first. The ground was still clutching its head and clambered onto the dungeon door.
"Harry," said Ron, looking for a moment she had ever seen a good death in the room and off the boys in the morning. He could not see him when he did not really dead it was the beast with a man who was sorting and scrambled over the table and pulled him on the ground and uncomfortable that had been straight to the castle. The portrait of the last word we had just entered the garden at the time was seemed to have forced to see her from the thing and of course not to tell them the dementors at the time.
"He was the real one in the country were the moment or the moment he had said it was the sword of Gryffindor and he saw him as far as if he said it had been some of the few weeks of the force of a goat of red eyes and the more spells when he had never seen getting the elf to the orphanage. He could hardly make a sudden gold and delighted was a broom and pulled out the bed for a